<a href="https://colab.research.google.com/github/GergelyTuri/pupillometry-rig/blob/main/notebooks/pupil_07_03_GT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pupil Tracking notebook. Based on Amelie's setup.
* 9/29/2023. used by GT. No Training required only the `Start Analyzing` section. It worked pretty well with a few exceptions. Notes are in `data_to_analyze` spreadsheet. Deeplabcut version: 2.3.5.


# DeepLabCut Toolbox - Colab
https://github.com/AlexEMG/DeepLabCut

This notebook illustrates how to use the cloud to:
- create a training set
- train a network
- evaluate a network
- create simple quality check plots
- analyze novel videos!

###This notebook assumes you already have a project folder with labeled data!

This notebook demonstrates the necessary steps to use DeepLabCut for your own project.

This shows the most simple code to do so, but many of the functions have additional features, so please check out the overview & the protocol paper!

Nath\*, Mathis\* et al.: Using DeepLabCut for markerless pose estimation during behavior across species. Nature Protocols, 2019.


Paper: https://www.nature.com/articles/s41596-019-0176-0

Pre-print: https://www.biorxiv.org/content/biorxiv/early/2018/11/24/476531.full.pdf


## First, go to "Runtime" ->"change runtime type"-> and then make sure "GPU" is selected


In [ ]:
#(this will take a few minutes to install all the dependences!)
!pip install deeplabcut[tf]

**(Be sure to click "RESTART RUNTIME" is it is displayed above above before moving on !)**

In [ ]:
# Use TensorFlow 1.x:
# %tensorflow_version 2

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


## Link your Google Drive (with your labeled data, or the demo data):

### First, place your porject folder into you google drive! "i.e. move the folder named "Project-YourName-TheDate" into google drive.

In [ ]:
#Now, let's link to your GoogleDrive. Run this cell and follow the authorization instructions:
#(We recommend putting a copy of the github repo in your google drive if you are using the demo "examples")

from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


YOU WILL NEED TO EDIT THE PROJECT PATH **in the config.yaml file** TO BE SET TO YOUR GOOGLE DRIVE LINK!

Typically, this will be: /content/drive/My Drive/yourProjectFolderName


In [ ]:
#Setup your project variables:
# PLEASE EDIT THESE:

ProjectFolderName = 'pupil-gergo-2021-07-03'
VideoType = 'avi'
Behavior = 'pupil' #name of model folder in the drive
videofile_path = ['/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert46b_5_sal_2022-09-01-143245-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert46b_5_pcb_2022-09-01-150642-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert47a_1_pcb_2022-09-01-120714-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert47a_1_sal_2022-09-01-113610-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/126031_5_bsline2_2022-12-09-131725-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/126031_5_pcb01_2022-12-09-135807-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/126031_5_pcb02_2022-12-09-143202-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/sert46b_5_sal_2022-09-01-143245-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/pv_r_baseline_FOV1_2023-09-19-102905-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/pv_r_baseline_FOV2_2023-09-19-104600-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/pv_r_pcb1_FOV1_2023-09-19-111813-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/pv_r_pcb1_FOV2_2023-09-19-110606-0000.avi',
                  '/content/drive/Shareddrives/Turi_lab/Data/psilocybin_project/PCB_pupil/pv_r_pcb1_FOV2_2023-09-19-115353-0000.avi'
                  ] #Enter the list of videos or folder to analyze.
                  # '/content/drive/MyDrive/DLC/samlple_data/pupil_test/5ht2afl05b_1_short.avi'
                  # '/content/drive/MyDrive/DLC/samlple_data/pupil_test/5ht2afl05b_1_2021-07-02-104829-0000.avi'
videofile_path

In [ ]:
#GUIs don't work on the cloud, so label your data locally on your computer! This will suppress the GUI support
import os
os.environ["DLClight"]="True"

In [ ]:
import deeplabcut

Loading DLC 2.3.6...
DLC loaded in light mode; you cannot use any GUI (labeling, relabeling and standalone GUI)


In [ ]:
deeplabcut.__version__

'2.3.5'

In [ ]:
#This creates a path variable that links to your google drive copy
#No need to edit this, as you set it up before:
path_config_file = '/content/drive/Shareddrives/Turi_lab/DeepLabCut/DLC_model_training/Pupil/Pupil -Amelie Wu-2022-04-01/config.yaml'

path_config_file

'/content/drive/Shareddrives/Turi_lab/DeepLabCut/DLC_model_training/Pupil/Pupil -Amelie Wu-2022-04-01/config.yaml'

## Create a training dataset:
### You must do this step inside of Colab:
After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
# Note: if you are using the demo data (i.e. examples/Reaching-Mackenzie-2018-08-30/), first delete the folder called dlc-models!
#Then, run this cell. There are many more functions you can set here, including which netowkr to use!
#check the docstring for full options you can do!
deeplabcut.create_training_dataset(path_config_file, net_type='resnet_50', augmenter_type='imgaug')

## Start training:
This function trains the network for a specific shuffle of the training dataset.

In [ ]:
#let's also change the display and save_iters just in case Colab takes away the GPU...
#if that happens, you can reload from a saved point. Typically, you want to train to 200,000 + iterations.
#more info and there are more things you can set: https://github.com/AlexEMG/DeepLabCut/blob/master/docs/functionDetails.md#g-train-the-network

deeplabcut.train_network(path_config_file, shuffle=1, displayiters=10,saveiters=500)

#this will run until you stop it (CTRL+C), or hit "STOP" icon, or when it hits the end (default, 1.03M iterations).
#Whichever you chose, you will see what looks like an error message, but it's not an error - don't worry....

**When you hit "STOP" you will get a KeyInterrupt "error"! No worries! :)**

## Start evaluating:
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
%matplotlib notebook
deeplabcut.evaluate_network(path_config_file,plotting=True)

# Here you want to see a low pixel error! Of course, it can only be as good as the labeler,
#so be sure your labels are good! (And you have trained enough ;)

## There is an optional refinement step you can do outside of Colab:
- if your pixel errors are not low enough, please check out the protocol guide on how to refine your network!
- You will need to adjust the labels **outside of Colab!** We recommend coming back to train and analyze videos...
- Please see the repo and protocol instructions on how to refine your data!

## Start Analyzing videos:
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides.

In [ ]:
deeplabcut.analyze_videos(path_config_file,videofile_path, videotype=VideoType)

## Plot the trajectories of the analyzed videos:
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
# this didn't work
deeplabcut.plot_trajectories(path_config_file,videofile_path, videotype=VideoType)

Now you can look at the plot-poses file and check the "plot-likelihood.png" might want to change the "p-cutoff" in the config.yaml file so that you have only high confidnece points plotted in the video. i.e. ~0.8 or 0.9. The current default is 0.4.

## Create labeled video:
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides.

In [ ]:
deeplabcut.create_labeled_video(path_config_file,videofile_path, videotype=VideoType)